# Testing Prompts for generating synthetic data

## Importing libraries

In [1]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM

import os
import pandas as pd

# Set Pandas to display all columns
pd.set_option('display.max_columns', None)

In [2]:
import os
import sys
from pathlib import Path

path = Path(os.path.dirname(os.getcwd()))
path = str(path)
print(path)
sys.path.insert(1, path)

/Users/saideepbunny/Projects/Application_Ranking_System


## Reading the data

In [3]:
df = pd.read_csv(f"{path}/data/job_descriptions.csv")
df

,Job Id,Experience,Qualifications,Salary Range,location,Country,latitude,longitude,Work Type,Company Size,Job Posting Date,Preference,Contact Person,Contact,Job Title,Role,Job Portal,Job Description,Benefits,skills,Responsibilities,Company,Company Profile
0,1089843540111562,5 to 15 Years,M.Tech,$59K-$99K,Douglas,Isle of Man,54.2361,-4.5481,Intern,26801,2022-04-24,Female,Brandon Cunningham,001-381-930-7517x737,Digital Marketing Specialist,Social Media Manager,Snagajob,Social Media Managers oversee an organizations...,"{'Flexible Spending Accounts (FSAs), Relocatio...","Social media platforms (e.g., Facebook, Twitte...","Manage and grow social media accounts, create ...",Icahn Enterprises,"{""Sector"":""Diversified"",""Industry"":""Diversifie..."
1,398454096642776,2 to 12 Years,BCA,$56K-$116K,Ashgabat,Turkmenistan,38.9697,59.5563,Intern,100340,2022-12-19,Female,Francisco Larsen,461-509-4216,Web Developer,Frontend Web Developer,Idealist,Frontend Web Developers design and implement u...,"{'Health Insurance, Retirement Plans, Paid Tim...","HTML, CSS, JavaScript Frontend frameworks (e.g...","Design and code user interfaces for websites, ...",PNC Financial Services Group,"{""Sector"":""Financial Services"",""Industry"":""Com..."
2,481640072963533,0 to 12 Years,PhD,$61K-$104K,Macao,"Macao SAR, China",22.1987,113.5439,Temporary,84525,2022-09-14,Male,Gary Gibson,9687619505,Operations Manager,Quality Control Manager,Jobs2Careers,Quality Control Managers establish and enforce...,"{'Legal Assistance, Bonuses and Incentive Prog...",Quality control processes and methodologies St...,Establish and enforce quality control standard...,United Services Automobile Assn.,"{""Sector"":""Insurance"",""Industry"":""Insurance: P..."
3,688192671473044,4 to 11 Years,PhD,$65K-$91K,Porto-Novo,Benin,9.3077,2.3158,Full-Time,129896,2023-02-25,Female,Joy Lucero,+1-820-643-5431x47576,Network Engineer,Wireless Network Engineer,FlexJobs,"Wireless Network Engineers design, implement, ...","{'Transportation Benefits, Professional Develo...",Wireless network design and architecture Wi-Fi...,"Design, configure, and optimize wireless netwo...",Hess,"{""Sector"":""Energy"",""Industry"":""Mining, Crude-O..."
4,117057806156508,1 to 12 Years,MBA,$64K-$87K,Santiago,Chile,-35.6751,-71.5429,Intern,53944,2022-10-11,Female,Julie Johnson,343.975.4702x9340,Event Manager,Conference Manager,Jobs2Careers,A Conference Manager coordinates and manages c...,"{'Flexible Spending Accounts (FSAs), Relocatio...",Event planning Conference logistics Budget man...,Specialize in conference and convention planni...,Cairn Energy,"{""Sector"":""Energy"",""Industry"":""Energy - Oil & ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1615935,134563577088850,0 to 12 Years,B.Tech,$64K-$114K,"Malabo (de jure),",Equatorial Guinea,1.6508,10.2679,Full-Time,18281,2022-05-19,Both,Shawn Cortez,950-451-5843,Mechanical Engineer,Mechanical Design Engineer,ZipRecruiter,Mechanical Design Engineers create and develop...,"{'Employee Assistance Programs (EAP), Tuition ...","Mechanical engineering CAD software (e.g., Sol...","Design mechanical systems, components, and pro...",The Hershey Company,"{""Sector"":""Food and Beverage/Confectionery"",""I..."
1615936,618604818190827,2 to 14 Years,M.Tech,$62K-$130K,Warsaw,Poland,51.9194,19.1451,Intern,63621,2023-03-14,Male,Steven Diaz,676.387.1572x71877,IT Manager,IT Director,USAJOBS,An IT Director oversees an organizations IT de...,"{'Health Insurance, Retirement Plans, Paid Tim...",Strategic IT planning Leadership and managemen...,Provide strategic leadership for IT department...,EQT,"{""Sector"":""Energy"",""Industry"":""Energy"",""City"":..."
1615937,615471367712200,4 to 15 Years,BCA,$60K-$96K,Ashgabat,Turkmenistan,38.9697,59.5563,Part-Time,114287,2022-01-23,Female,Bryan Morales,537.384.6193x5284,Mechanical Engineer,Mechanical Design Engineer,Indeed,Mechanical Design Engineers create and develop...,"{'Tuition Reimbursemen

## Testing prompts

In [4]:
i=1

In [5]:
print("**Job Role:**", df.iloc[i]["Role"])
print("**Experience:**", df.iloc[i]["Experience"])
print("**Qualifications:**", df.iloc[i]["Qualifications"])
print("**Salary Range:**", df.iloc[i]["Salary Range"])
print("**Company:**", df.iloc[i]["Company"])
print("**Company Profile:**", df.iloc[i]["Company Profile"])

**Job Role:** Frontend Web Developer
**Experience:** 2 to 12 Years
**Qualifications:** BCA
**Salary Range:** $56K-$116K
**Company:** PNC Financial Services Group
**Company Profile:** {"Sector":"Financial Services","Industry":"Commercial Banks","City":"Pittsburgh","State":"Pennsylvania","Zip":"15222","Website":"www.pnc.com","Ticker":"PNC","CEO":"William S. Demchak"}


### Generating elaborated Job Description

In [6]:
jd_template = f"""
You are a creative and experienced job recruiter tasked with crafting a **detailed, engaging, and relevant job description** that aligns with the provided job role, required experience, qualification, salary range, and company profile. Your goal is to produce a description that resonates with potential candidates while highlighting the unique aspects of the role and organization.

### **Input Details**:
- **Job Role**: A concise summary of the role associated with the job position.
- **Years of Experience**: A range (0 to 16 years) specifying the required experience level.
- **Qualification Required**: The minimum qualification required for the job.
- **Salary Range**: The range of salary offered for the job, which should influence the complexity of the description.
- **Company Profile**: A brief overview of the company's vision, mission, and culture.

### **Instructions**:
- Write a **detailed and compelling job description** containing **50 to 250 words**.
- Ensure the description aligns with the provided **years of experience** and **salary range**:
  - For higher experience levels, salaries or qualification, emphasize strategic responsibilities, leadership opportunities, or advanced expertise.
  - For entry-level roles or lower salary ranges, focus on growth potential, mentorship, and foundational contributions.
- Integrate the **Job Role, Qualification Required, and Company Profile** seamlessly into the description to make it appealing and relevant.
- Add creative elements that reflect the company’s unique culture and vision while maintaining professionalism.
- Avoid using generic language; instead, tailor the description to the specific inputs provided.
- **Do not include additional sections** such as "Responsibilities," "Skills," or "Qualifications." Focus exclusively on crafting the job description.

### **Input**:
**Job Role**: {{job_role}}  
**Years of Experience**: {{exp_req}}  
**Qualification Required**: {{qualification}}  
**Salary Range**: {{salary}}  
**Company Profile**: {{profile}}  

### **Output**:
Generate only the **detailed, relevant, and creative job description**. Avoid any additional text, headings, or formatting outside of the description itself.
"""

In [7]:
model = OllamaLLM(model="llama3.2", temperature = 1)

prompt = ChatPromptTemplate.from_template(jd_template)
chain = prompt | model
data = {
    "job_role": df.iloc[i]["Role"],
    "exp_req": df.iloc[i]["Experience"],
    "qualification": df.iloc[i]["Qualifications"],
    "salary": df.iloc[i]["Salary Range"],
    "profile": df.iloc[i]["Company Profile"]
}
jd = chain.invoke(data)

In [8]:
print(jd)

As a Frontend Web Developer at PNC, you will be part of a talented team that leverages cutting-edge technology to drive business growth in Pittsburgh's financial services sector. With a strong educational background in Business and Computer Applications (BCA) and 2-12 years of experience in developing dynamic web applications, you'll play a crucial role in shaping the online presence of one of America's most iconic banks.

In this exciting opportunity, you will focus on crafting visually appealing user interfaces, implementing responsive designs, and ensuring seamless cross-browser compatibility. Your technical expertise will be leveraged to develop innovative digital solutions that enhance customer engagement, streamline processes, and drive efficiency.

As a strategic partner in our commercial banking division, you will have opportunities to collaborate with cross-functional teams, share knowledge, and contribute to the evolution of PNC's web development practices. With a salary rang

### Generating skills

In [9]:
s_template = f"""
You are a professional job recruiter tasked with creating a **comprehensive and precise list of skills** required for a specific job role. Your goal is to generate a detailed set of skills based on the provided job role, years of experience, qualifications, salary range, company profile, and job description.

### **Input Details**:
- **Job Role**: A concise summary of the role associated with the job position.
- **Years of Experience**: A range (0 to 16 years) specifying the required experience level.
- **Qualification Required**: The minimum qualification required for the job.
- **Salary Range**: The range of salary offered for the job, which should influence the complexity and expertise of required skills.
- **Company Profile**: A brief overview of the company's vision, mission, culture, and work environment.
- **Job Description**: A detailed description of the job role, outlining its purpose and expectations.

### **Instructions**:
- Generate a **comma-separated list** of skills relevant to the job. The skills must be categorized but presented without headings, as follows:
  - **Soft Skills**: Workplace and interpersonal skills critical for the role (e.g., communication, problem-solving, adaptability).
  - **Technical Skills**: Domain-specific abilities or knowledge areas essential for job success (e.g., programming, data analysis, system troubleshooting).
  - **Technology Skills**: Tools, software, frameworks, or platforms commonly used for the job (e.g., cloud platforms, databases, development tools).
- Ensure the output includes:
  - **10 to 15 unique soft skills** that are relevant to the job role and aligned with the years of experience.
  - **5 to 10 unique technical skills** tailored to the job requirements and expertise needed.
  - **10 to 15 unique technology-related skills** directly applicable to tools and platforms specified in the job description.
- Ensure that the skills are:
  - **Relevant**: Clearly aligned with the provided job role and inputs.
  - **Non-redundant**: Avoid repeating the same skill across different categories.
  - **Diverse**: Covering a broad range of capabilities suited to the role, experience, and salary range.
- Write the output as a **single comma-separated list of skills**. 
- Do **not include headings**, additional text, or formatting outside of the skills list.

### **Input**:
**Job Role**: {{job_role}}  
**Years of Experience**: {{exp_req}}  
**Qualification Required**: {{qualification}}  
**Salary Range**: {{salary}}  
**Company Profile**: {{profile}}  
**Job Description**: {{jd}}

### **Output**:
Generate only a **comma-separated list of relevant skills** satisfying the given constraints. Do not include any additional text, headings, or formatting outside of the list.
"""


In [10]:
model = OllamaLLM(model="llama3.2", temperature = 1)

prompt = ChatPromptTemplate.from_template(s_template)
chain = prompt | model
data = {
    "job_role": df.iloc[i]["Role"],
    "exp_req": df.iloc[i]["Experience"],
    "qualification": df.iloc[i]["Qualifications"],
    "salary": df.iloc[i]["Salary Range"],
    "profile": df.iloc[i]["Company Profile"],
    "jd": jd
}
skills = chain.invoke(data)

In [11]:
print(skills)

Soft Skills: communication, problem-solving, adaptability, teamwork, time management, leadership, customer service, critical thinking, creativity, innovation
Technical Skills: front-end development, responsive design, cross-browser compatibility, HTML5, CSS3, JavaScript, web application development, server-side programming, database integration, testing and debugging
Technology Skills: React.js, Node.js, MongoDB, PostgreSQL, HTML5 Canvas, CSS Grid, Webpack, Babel, Bootstrap


### Generating Job responsibilities

In [12]:
jr_template = f"""
You are a professional job recruiter tasked with generating a detailed, precise, and structured list of **key job responsibilities** for a specific role. Your goal is to create a clear and actionable description that aligns with the job role, years of experience, qualifications, salary range, company profile, job description, and skills.

### **Input Details**:
- **Job Role**: A concise summary of the role associated with the job position.
- **Years of Experience**: A range (0 to 16 years) specifying the required experience level.
- **Qualification Required**: The minimum qualification required for the job.
- **Salary Range**: The range of salary offered for the job, which should influence the complexity and expectations of responsibilities.
- **Company Profile**: A brief overview of the company's vision, mission, and culture.
- **Job Description**: A detailed overview of the job role, its purpose, and primary expectations.
- **Skills**: A comma-separated list of both technical and soft skills relevant to the role.

### **Instructions**:
- Generate a **structured, detailed list of 8 to 15 bullet points** describing the **core job responsibilities**.
- Ensure that the responsibilities include:
  - **Experience-based requirements**: Start with high-level expectations for years of professional experience and specific qualifications.
  - **Technical expertise**: Highlight responsibilities that reflect the provided technical and technology-related skills.
  - **Role-specific tasks**: Describe actionable, measurable, and impactful responsibilities aligned with the job role and description.
  - **Soft skills and collaboration**: Integrate soft skills (e.g., communication, problem-solving) into responsibilities where relevant.
- **Ensure diversity** in the responsibilities to cover different facets of the role, such as technical tasks, strategic goals, collaboration, troubleshooting, optimization, and stakeholder communication.
- **Align complexity and depth** of the responsibilities with the provided **years of experience** and **salary range**:
  - For higher experience and salary levels, emphasize leadership, strategic initiatives, and complex tasks.
  - For lower experience and salary levels, focus on learning opportunities, foundational contributions, and growth potential.
- Write each responsibility in **clear, concise, and professional sentences** for easy readability.

### **Input**:
**Job Role**: {{job_role}}  
**Years of Experience**: {{exp_req}}  
**Qualification Required**: {{qualification}}  
**Salary Range**: {{salary}}  
**Company Profile**: {{profile}}  
**Job Description**: {{jd}}  
**Skills**: {{skills}}

### **Output**:
Provide a structured, detailed list of job responsibilities as bullet points. Avoid including any extra text, headings, or formatting outside of the list. The tone should be professional, detailed, and engaging.
"""


In [13]:
model = OllamaLLM(model="llama3.2", temperature = 1)

prompt = ChatPromptTemplate.from_template(jr_template)
chain = prompt | model
data = {
    "job_role": df.iloc[i]["Role"],
    "exp_req": df.iloc[i]["Experience"],
    "qualification": df.iloc[i]["Qualifications"],
    "salary": df.iloc[i]["Salary Range"],
    "profile": df.iloc[i]["Company Profile"],
    "jd": jd,
    "skills":skills
}
jr = chain.invoke(data)

In [14]:
print(jr)

• Develop visually appealing user interfaces with a focus on responsive design and cross-browser compatibility for PNC's commercial banking division website and other digital platforms.
• Implement dynamic web applications using HTML5, CSS3, JavaScript, and server-side programming languages such as Node.js to enhance customer engagement and streamline processes.
• Ensure seamless integration of database systems like MongoDB and PostgreSQL to store and retrieve data efficiently.
• Collaborate with cross-functional teams to share knowledge, contribute to the evolution of PNC's web development practices, and drive business growth in Pittsburgh's financial services sector.
• Troubleshoot and resolve technical issues related to front-end development, server-side programming, and database integration to ensure high-quality digital solutions.
• Optimize code efficiency using tools such as Webpack, Babel, and Bootstrap to improve application performance and scalability.
• Participate in testin

### Generating Resume

In [15]:
category_requirements = {
    "complete_mismatch": """
    Resume is not at all a fit for job description. Below are some of the constraints: 
        - Use a completely different career path.
        - Include skills from a different domain.
        - Maintain professional quality despite the mismatch.
        - Other fields like experience, highest qualification do not matter here as the career path is completely different.
    """,
    "underwhelming": """
    Resume falls short in certain aspects and is an underwhelming for job description. Below are some of the constraints: 
        - Include relevant field but insufficient experience.
        - Miss some critical required skills.
        - Include partial qualification matches.
        - Show insufficient project experience.
    """,
     "good": """
     Resume is a perfect for job description. Below are some of the constraints: 
        - Match all required qualifications.
        - Include relevant project experience or work experience.
        - Show appropriate skill level with almost 95%-100% skill match.
        - Demonstrate progression in career.
        - Bullet points for Projects or Work experience section must include APR format. Action + Problem solved + Result.
    """,
    "overqualified": """
    Resume is over qualified for job description. Below are some of the constraints: 
        - Exceed required experience by 3-5 years.
        - Show higher level skills, qualifications and responsibilities.
        - Include additional advanced certifications, work experience or project work.
        - Demonstrate a progressive and over-qualified career.
        - Bullet points for Projects or Work experience section must include APR format. Action + Problem solved + Result.
    """
}

In [16]:
# Define the template
resume_template = """
You are a professional resume writer tasked with generating a resume based on the following inputs and requirements:

## **Job Details**:
- Role: {job_role}
- Experience required: {exp_req}
- Required Education: {qualification}
- Job Description: {jd}
- Job Responsibilities: {jr}
- Required Skills: {skills}


Fit Category: {fit_category}
Category Requirements:
{category_requirements}

## **Instructions**:
1. Generate a professional resume that matches the specified fit category ({fit_category}) while adhering to its constraints.
2. The resume must be 500-700 words and include these mandatory sections:
   - Education
   - Skills
   - Work Experience (If no experience, compensate with robust Projects section)

3. Include relevant optional sections based on the context:
   - Professional Summary
   - Projects (Required if limited work experience; Optional if extensive experience)
   - Certifications/Accomplishments
   - Other relevant sections as needed

4. Guidelines for Section information:
    - Education : Write University name, degree name, major name and gpa
    - Skills : Soft skills, technical skills, technologies, frameworks etc., separated by comma
    - Work Experience : Company name, role, Part-time or full-time, number of  years worked and the respective bullet points.
    - Professional Summary : Professional summary of the candidate
    - Projects : Project name, and their respective bullets describing what they did.
    - Certifications : list of certifications and accomplishments

5. Format Guidelines:
   - Use clear section headings
   - Maintain professional formatting
   - Ensure content aligns with fit category requirements
   - Balance between work experience and projects based on the role and fit category wherever applicable.

6. Specific Requirements:
   - For complete_mismatch: Create a professional resume from a different career path. Should be 0-30% match to the Job details.
   - For underwhelming: Include relevant but insufficient experience/skills. Should be 50-70% match to the Job details.
   - For good_fit: Match 95-100% of requirements with appropriate progression.
   - For overqualified: Exceed requirements with advanced qualifications.

## **Output**:
Generate a professional resume following these guidelines while maintaining authenticity and relevance adhering to all the constraints given. Avoid including any extra text, headings, or formatting outside of what is required.
"""


In [20]:
model = OllamaLLM(model="llama3.2", temperature = 1)

prompt = ChatPromptTemplate.from_template(resume_template)
chain = prompt | model
data = {
    "job_role": df.iloc[i]["Role"],
    "exp_req": df.iloc[i]["Experience"],
    "qualification": df.iloc[i]["Qualifications"],
    "jd": jd,
    "jr":jr,
    "skills":skills,
    "fit_category": "Good",
    "category_requirements": category_requirements["good"]
}
resume = chain.invoke(data)

In [21]:
print(resume)

**Full Name:** Rohan D. Patel
**Contact Information:**

- Phone: +1 123 456 7890
- Email: rohan.patel@email.com
- LinkedIn Profile: linkedin.com/in/rohandompattel
- Address: Pittsburgh, PA, USA

**Professional Summary:**
Highly skilled Frontend Web Developer with a strong educational background in Business and Computer Applications (BCA) and 8 years of experience developing dynamic web applications. Proficient in a range of front-end development technologies including HTML5, CSS3, JavaScript, React.js, Node.js, MongoDB, PostgreSQL, and Bootstrap. Proven track record of delivering high-quality digital solutions that enhance customer engagement and drive business growth.

**Education:**

- **Bachelor of Computer Applications (BCA), Carnegie Mellon University**, Pittsburgh, PA, USA
    • Date: 2009 - 2013
    • GPA: 3.5/4.0
    • Major: Business and Computer Applications

**Skills:**

Soft Skills:
Communication, Problem-Solving, Adaptability, Teamwork, Time Management, Leadership, Custome